### 1.SparkSQL 對Python, Java和Scala語言提供了DataFrame框架,這類似關聯式資料庫的表格,可簡化處理結構化資料即時的工作
### 2.SparkSQL can do the 結構化資料 的讀和寫
### 3.Can let Users use SQL to query data

In [1]:
### SparkSQL is base on RDD where is call DataFrame. It is clude row.

### connect to Spark SQL

In [4]:
from pyspark.sql import SQLContext, Row
sc = SparkContext()

In [5]:
sqlContext = SQLContext(sc)

In [6]:
input = sqlContext.read.json("/home/jordan/Desktop/testTweets")

In [7]:
## is very import to register table
input.registerTempTable("testTweets")

In [10]:
topTweets = sqlContext \
.sql("""select text, retweetCount from testTweets \
order by retweetCount limit 10""")

In [21]:
topTweets.show()

+--------------------+------------+
|                text|retweetCount|
+--------------------+------------+
|Adventures With C...|           0|
+--------------------+------------+



In [42]:
Tweets = sqlContext.sql("""select * from testTweets""")

In [43]:
Tweets.show()

+---------------+--------------------+--------------------+---------------+------------------+-----------------+---------------+-----------+-------------------+-----------+-------------+------------+--------------------+--------------------+-----------+--------------------+-------------------+
|contributorsIDs|           createdAt|currentUserRetweetId|hashtagEntities|                id|inReplyToStatusId|inReplyToUserId|isFavorited|isPossiblySensitive|isTruncated|mediaEntities|retweetCount|              source|                text|urlEntities|                user|userMentionEntities|
+---------------+--------------------+--------------------+---------------+------------------+-----------------+---------------+-----------+-------------------+-----------+-------------+------------+--------------------+--------------------+-----------+--------------------+-------------------+
|             []|Nov 4, 2014 4:56:...|                  -1|             []|529799371026485248|               -1|   

In [47]:
Tweets.select("source").show()

+--------------------+
|              source|
+--------------------+
|<a href="http://t...|
+--------------------+



In [ ]:
Tweets.filter(df("age") > 19)
Tweets.groupBy(df("age")).max()

### DataFrames to RDD

In [51]:
topTweetRDD = topTweets.rdd.map(lambda row: row.text)
topTweetRDD

PythonRDD[64] at RDD at PythonRDD.scala:48

### Cache =>usually use to query same table was save in (in-memory)

In [ ]:
SQLContext.cache()

### Parquet is column-orient儲存格式,有效率的儲存在巢狀欄位結構,因此常用於Hadoop各類專案中,並且支援SparkSQL中所有類別型態

In [59]:
#load Parquet
rows = sqlContext.load(parquetFile, "parquet")
names = rows.map(lambda row: row.name)
print "Everyone"
print names.collect

In [ ]:
#query Parquet
tb1 = rows.registerTempTable("people")
pandaFriends = sqlContext.sql("select name from people where favotiteAnimal = \"panda\"")
print "Panda friends"
print pandaFriends.map(lambda row: row.name).collect()

In [ ]:
#sav Parquet
pandaFriends.save"hdfs://...", "parquet")

### JSON

In [70]:
inputs = sqlContext.read.json("/home/jordan/Desktop/testTweets")

### Build DataFrame from RDD

In [72]:
happyPeopleRDD = sc.parallelize([Row(name="holden", favouriteBeverage="coffee")])

In [76]:
happyPeopleDataFrame = sqlContext.createDataFrame(happyPeopleRDD)

In [77]:
happyPeopleDataFrame

DataFrame[favouriteBeverage: string, name: string]

In [78]:
happyPeopleDataFrame.registerTempTable("happy_people")

### JDBC server

In [ ]:
JDBC is base on sbin/start-thriftserver.sh
is localhost:10000


### 可以透過Beeline客戶端連結至JDBC伺服器


In [ ]:
./bin/beeline -u jdbc:hive2://localhost:1000
            
> show tables;

### Spark SQL UDF

In [ ]:
from pyspark.sql.types import IntegerType

hiveCtx.registerFunction("strLenPython", lambda x: len(x), IntegerType())
lengthDataFrame = hiveCtx.sql("select strLenPython('text) from tweets limit 10")
lengthDataFrame.show()

In [ ]:
當資料被cache(),sparkSQL 使用 in-memory 
的column-based儲存方法,占用較少的記憶體

In [80]:
sc.stop()